In [31]:
import bs4
import requests
import pandas as pd
import numpy as np
import time
import os
from IPython.display import clear_output

In [32]:
# 요청 함수
def getSource(site, start) :
    # 해더 정보 셋팅
    header_info = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36',
        'X-Naver-Client-Id' : 'opLaOk6efaoqdcpG3V8j',
        'X-Naver-Client-Secret' : 'nvzTR4HuHt',
    }
    
    # 파라미터 데이터
    param_info = {
        'query' : '코로나',
        'display' : 100,
        'start' : start,
        'sort' : 'sim',
    }
    # 요청한다.
    response = requests.get(site, headers=header_info, params=param_info)
    return response.text

In [33]:
# 한 페이지의 데이터를 가져오는 함수
def getData(source) :
    # bs4 객체를 생성한다.
    soup = bs4.BeautifulSoup(source)
    
    # item 태그들을 가져온다.
    item_list = soup.select('item')
    
    data_dict = {
        'title' : [],
        'description' : [],
        'bloggername' : [],
        'bloggerlink' : [],
        'postdate' : [],
    }
    
    for item_tag in item_list :
        # print(item_tag)
        
        # title
        a1 = item_tag.select_one('title')
        data1 = getText(a1)
        data1 = removeChar(data1)
        
        # description
        a2 = item_tag.select_one('description')
        data2 = getText(a2)
        data2 = removeChar(data2)
        
        # bloggername
        a3 = item_tag.select_one('bloggername')
        data3 = getText(a3)
        data3 = removeChar(data3)
        
        # bloggerlink
        a4 = item_tag.select_one('bloggerlink')
        data4 = getText(a4)
        
        # postdate
        a5 = item_tag.select_one('postdate')
        data5 = getText(a5)
        
        # print(data1)
        # print(data2)
        # print(data3)
        # print(data4)
        # print(data5)
        # print('--------------------------')
                
        # 데이터를 담는다.
        data_dict['title'].append(data1)
        data_dict['description'].append(data2)
        data_dict['bloggername'].append(data3)
        data_dict['bloggerlink'].append(data4)
        data_dict['postdate'].append(data5)
        
    df1 = pd.DataFrame(data_dict)
    if os.path.exists('./09_data.csv') == False :
        df1.to_csv('./09_data.csv', encoding='utf-8-sig', index=False)
    else :
        df1.to_csv('./09_data.csv', encoding='utf-8-sig', index=False, header=None, mode='a')

In [34]:
# 문자열 내에서 웹 특수문자를 제거하는 함수
# 다른 것들이 더 있으면 같은 방식으로 작성해주세요
def removeChar(text) :
    # if pd.notna(text) :
    if pd.isna(text) == True :
        return text
    text = text.replace('<b>', '')
    text = text.replace('</b>', '')
    text = text.replace('&quot;', '"')
    text = text.replace('&lt;', '<')
    text = text.replace('&gt;', '>')
    text = text.replace('&apos;', "'")
    text = text.replace('&amp;', "&")
    return text

In [35]:
# 전달된 태그로 부터 문자열 데이터를 추출해 반환하는 함수
def getText(tag):
    # 태그가 없다면
    if tag == None :
        return np.nan
    # 문자열을 가져온다.
    data = tag.text.strip()
    # 길이가 0 이라면...
    if len(data) == 0 :
        return np.nan
    return data

In [36]:
site = 'https://openapi.naver.com/v1/search/blog.xml'

start = 1
# 네이버 오픈 api 에서 start는 1000이 최대므로...
while start < 1000 :
    time.sleep(1)
    clear_output(wait=True)
    
    print(f'{start} 수집중')
    
    # 요청한다.
    source = getSource(site, start)
    # print(source)
    
    # 한 페이지의 데이터를 가져온다.
    getData(source)
    
    # start를 100 증가시킨다.
    start = start + 100
print('수집완료')

901 수집중
수집완료
